In [2]:
#adaptar el algoritme al dato, busque altre modelo
#enotoin detection cnn
#captura imagen
import pandas as pd
import numpy as np
import tensorflow as tf
import cv2

# Cargar el conjunto de datos fer2013, fuente kaggle
data = pd.read_csv('./data/fer2013.csv')

# Preparar las imágenes y etiquetas
imagenes = []
etiquetas = data['emotion'].values

#Redimensionar las imagenes para que sean 48x48 pixeles
for img in data['pixels']:
    imagen = np.fromstring(img, dtype=int, sep=' ').reshape(48, 48)
    imagenes.append(imagen)

#normalizar las imagenes redimensionadas
imagenes = np.array(imagenes).astype('float32') / 255.0
imagenes = np.expand_dims(imagenes, -1)  # Expandir dimensiones para Keras/TensorFlow

# Ahora 'imagenes' y 'etiquetas' están listos para ser utilizados en un modelo de aprendizaje automático


In [ ]:
from keras.applications.inception_v3 import preprocess_input

img_size = (299, 299)
num_classes = 7

def preprocess(image, label):
    image = tf.image.resize(image, img_size)
    image = tf.repeat(image, 3, axis=-1)
    image = tf.cast(image, tf.float32)
    image = preprocess_input(image)
    label = tf.one_hot(label, num_classes)
    return image, label


In [ ]:
# Crear un tf.data.Dataset a partir de tus arrays
dataset = tf.data.Dataset.from_tensor_slices((imagenes, etiquetas))


In [ ]:
# Aplicar la función de preprocesamiento a cada elemento del dataset
dataset = dataset.map(preprocess)


In [ ]:
train_size = int(0.6 * len(dataset))
val_size = int(0.2 * len(dataset))
test_size = len(dataset) - train_size - val_size
print(len(dataset))
train_dataset = dataset.take(train_size)
#cogemos a partir de train_size hasta una longitud de val_size
val_dataset = dataset.skip(train_size).take(val_size)
test_dataset = dataset.skip(train_size + val_size)
print(len(train_dataset), len(val_dataset), len(test_dataset))

In [ ]:


from keras.applications.inception_v3 import InceptionV3
from keras.layers import Input, GlobalAveragePooling2D, Dense, Dropout, RandomRotation, RandomTranslation, RandomZoom
from keras.models import Model

input_layer = Input(shape=img_size + (3,))
#capa de data aumentation, rotaciones, translaciones, etc.
x = RandomRotation(factor=0.1, fill_mode='nearest')(input_layer)
x = RandomTranslation(height_factor=0.1, width_factor=0.1, fill_mode='nearest')(x)
x = RandomZoom(height_factor=0.2, fill_mode='nearest')(x)


inception_model = InceptionV3(input_shape=img_size + (3,),include_top=False, weights='imagenet')

for layer in inception_model.layers:
    layer.trainable = False

x = inception_model(x)

x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(num_classes, activation='softmax')(x)

aug_model = Model(inputs=input_layer, outputs=output)

aug_model.summary()



In [ ]:
from keras.optimizers import Adam

opt=Adam(learning_rate=0.001)
aug_model.compile(loss='categorical_crossentropy',
            optimizer=opt,
            metrics=['accuracy'])



In [ ]:

from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from keras.models import load_model

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.00001)
checkpoint = ModelCheckpoint(filepath='best_model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)

epochs=4
batch_size=32
train_dataset_batched = train_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
val_dataset_batched = val_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
history = aug_model.fit(train_dataset_batched,
                    epochs=epochs,
                    verbose=1,
                    validation_data=val_dataset_batched,
                    callbacks=[reduce_lr,checkpoint])



In [ ]:


aug_model = load_model('best_model.h5')
test_dataset_batched = test_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
score = aug_model.evaluate(test_dataset_batched, verbose=0)
print(f'Test loss: {score[0]*100:.2f}')
print(f'Test accuracy: {score[1]*100:.2f}')

